In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import wikipedia
from tqdm import tqdm

In [68]:
def get_wiki(x):
    'get a wikipedia page from the title of an html snippet'
    return wikipedia.page(x.get('title'))
def get_wiki_html(x):
    return wikipedia.WikipediaPage(x.get('title'))

In [3]:
url_link = "https://en.wikipedia.org/wiki/Indigenous_languages_of_the_Americas"
content = requests.get(url_link).text
soup = BeautifulSoup(content,'html.parser')
table = soup.find_all('table', 'wikitable')

df= pd.read_html(str(table))
df=pd.DataFrame(df[0])

df = df.drop('Source',axis=1)
data = df.rename(columns={'Number of speakers':'number'})

In [4]:
def fixxr(x):
    x = x.split(' ',1)[0]
    x = x.replace(',','')
    return x

data.number=data.number.apply(fixxr)
#Remove abnormalities (beta)
for num in data.number:
    test = any(ele not in ['1','2','3','4','5','6','7','8','9','0'] for ele in list(num))
    if test== True:
        print(list(data.number).index(num))
data.number[90] = '10500'

data = data[data.number!= '0']
data.number = data.number.apply(int)
# data = data[data.number >= 10000]
# data['Number of speakers'].apply(.split(' ',1)[0])
data

90


,Language,number,Official Recognition,Area(s) Language is spoken
0,Guaraní,6500000,Paraguay (Official Language) Bolivia Corriente...,"Paraguay, Bolivia, Argentina, Brazil"
1,Southern Quechua,5000000,Bolivia (Official Language) Peru (Official Lan...,"Bolivia, Peru, Argentina, Chile"
2,Nahuatl,1700000,Mexico,Mexico
3,Aymara,1700000,Bolivia (Official Language) Peru (Official Lan...,"Bolivia, Peru, Chile"
4,Qʼeqchiʼ,1100000,Guatemala Belize Mexico,"Guatemala, Belize, Mexico"
...,...,...,...,...
229,Comanche,9,NaN,US
230,Carijona,6,"Colombia (Amazonas, Guaviare)","Colombia (Amazonas, Guaviare)"
231,Itonama,5,Bolivia,Bolivia
232,Kiliwa,4,Mexico,Mexico


In [44]:
table = soup.find('table', 'wikitable')
links = table.findAll('a')

keep = []
for link in links: # Remove all the non titles
    if 'title' in str(link):
        keep.append(link)
links = keep

keep = []
for link in tqdm(links):
    try:
        get_wiki(link)
        keep.append(link)
    except:
        pass
links = keep

# The problem at this point is that it deletes languages.
# Maybe I oculd use some metric of the categories and use ML to find similarties in the vectors...
keep = []
for link in tqdm(links): 
    if any('Languages of' in cat for cat in get_wiki(link).categories) == True:
        keep.append(link)
links = keep

# Everything Past this Point isn't complete

In [ ]:
# Get the content from the links
links_content = {}
print('Done extracting links. About to fetch: %s links..' % len(links))
for link in links:
    print('Fetching: %s' % link)
    links_content[link.get('title')] = get_wiki_html(link).html()

In [70]:
get_wiki_html(links[0]).html()

'<div class="mw-content-ltr mw-parser-output" lang="en" dir="ltr"><div class="shortdescription nomobile noexcerpt noprint searchaux" style="display:none">Indigenous language, Paraguay variant</div>\n<style data-mw-deduplicate="TemplateStyles:r1033289096">.mw-parser-output .hatnote{font-style:italic}.mw-parser-output div.hatnote{padding-left:1.6em;margin-bottom:0.5em}.mw-parser-output .hatnote i{font-style:normal}.mw-parser-output .hatnote+link+.hatnote{margin-top:-0.5em}</style><div role="note" class="hatnote navigation-not-searchable">This article is about the Paraguayan language. For other varieties of Guarani, see <a href="/wiki/Guarani_dialects" title="Guarani dialects">Guarani dialects</a>.</div>\n<link rel="mw-deduplicated-inline-style" href="mw-data:TemplateStyles:r1033289096"><div role="note" class="hatnote navigation-not-searchable">Not to be confused with the Kurdish-Iranian <a href="/wiki/Gorani_language" title="Gorani language">Gorani language</a>.</div>\n<style data-mw-ded

In [64]:
links[0].get('title')

'Guaraní language'

In [107]:
thing = get_wiki(links[56])
any('Languages of' in cat for cat in thing.categories)

False

In [41]:
any('Languages of' in cat for cat in thing.categories)

False